In [ ]:
# Import PyTorch
import torch

# Import os for file related activities and folder navigation
import os

# Import Pandas for data manipulation
import pandas as pd

# Import the Python Imaging Library (PIL) for image processing
from PIL import Image

# Import PyTorch torchvision.transforms for image transformations during training
import torchvision.transforms as transforms

# Import PyTorch torchvision.datasets and models
from torchvision import datasets, models

# Import NumPy for numerical operations
import numpy as np

import json

# Import the requests library for HTTP requests
import requests

import warnings
warnings.filterwarnings('ignore')

# Import PyTorch Dataset and DataLoader for data handling
from torch.utils.data import Dataset, DataLoader

# Import PyTorch functional and neural network libraries
import torch.nn.functional as F
import torch.nn as nn

# Import torchvision for additional image-related functionality
import torchvision

# Import SummaryWriter from torch.utils.tensorboard for TensorBoard logging
from torch.utils.tensorboard import SummaryWriter

# Import datetime and time for date and time operations
import datetime
import time

# Import specific components from torchvision.models.resnet
from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

# Clear GPU cache using torch.cuda.empty_cache()
torch.cuda.empty_cache()

# Determine the device (GPU or CPU) available for computation - ensure that cuda is displayed
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
## start the tensorboard
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
## Instantiate the writer object that is called during model training
writer = SummaryWriter(log_dir='logs')

2023-06-14 07:38:05.910798: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
## Check the working directory
os.getcwd()

'/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals'

In [ ]:
## Creating a train dataset class
class ItemsTrainDataSet(Dataset):
    """
    A custom dataset class for image training data.
    
    This class inherits from the PyTorch Dataset class and will 
    load and preprocess image training data.

    Attributes
    ----------
    None

    Methods
    -------
    __init__():
        Initializes a new ItemsTrainDataSet instance.
    _load_examples():
        Loads and prepares a list of image examples.
    __getitem__(idx):
        Retrieves an image and its associated class label by index.
    __len__():
        Returns the total number of examples in the dataset.
    """
    def __init__(self):
        """
        Initialize a new ItemsTrainDataSet instance.

        This constructor sets up the necessary transformations for image preprocessing.
        """        
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)

    def _load_examples(self):
        """
        Load and prepare a list of image examples.

        This method scans the training directory, assigns class labels,
        and collects a list of image file paths.

        Returns
        -------
        list
            A list of tuples containing image file paths and their class labels.
        """        
        class_names = os.listdir('pytorch_images_tv_split_2/train')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('pytorch_images_tv_split_2/train',cl_name))
            example_fp = [os.path.join('pytorch_images_tv_split_2/train', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        """
        Retrieve an image and its associated class label by index.

        Parameters
        ----------
        idx : int
            The index of the item to retrieve.

        Returns
        -------
        tuple
            A tuple containing the preprocessed image and its class label.
        """        
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)
        #features = self.rgbify(features)

        return features, img_class

    def __len__(self):
        """
        Return the total number of examples in the dataset.

        Returns
        -------
        int
            The total number of examples in the dataset.
        """        
        return len(self.examples)

In [ ]:
## Creates a validation dataset class
class ItemsValDataSet(Dataset):
    """
    A custom dataset class for image validation data.
    
    This class inherits from the PyTorch Dataset class and is designed
    for loading and preprocessing image validation data.

    Attributes
    ----------
    None

    Methods
    -------
    __init__():
        Initializes a new ItemsValDataSet instance.
    _load_examples():
        Loads and prepares a list of image examples.
    __getitem__(idx):
        Retrieves an image and its corresponding class label by index.
    __len__():
        Returns the total number of examples in the dataset.
    """
    def __init__(self):
        """
        Initialize a new ItemsValDataSet instance.

        This constructor sets up the necessary transformations for image preprocessing.
        """
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)!=1 else x)

    def _load_examples(self):
        """
        Load and prepare a list of image examples.

        This method scans the validation directory, assigns class labels,
        and collects a list of image file paths.

        Returns
        -------
        list
            A list of tuples containing image file paths and their class labels.
        """        
        class_names = os.listdir('pytorch_images_tv_split_2/val')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}
        examples_list = []

        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('pytorch_images_tv_split_2/val',cl_name))
            example_fp = [os.path.join('pytorch_images_tv_split_2/val', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        """
        Retrieve an image and its corresponding class label by index.

        Parameters
        ----------
        idx : int
            The index of the item to retrieve.

        Returns
        -------
        tuple
            A tuple containing the preprocessed image and its class label.
        """        
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)
        #features = self.rgbify(features)

        return features, img_class

    def __len__(self):
        """
        Return the total number of examples in the dataset.

        Returns
        -------
        int
            The total number of examples in the dataset.
        """        
        return len(self.examples)

In [ ]:
# Create the traindataset object 
traindataset = ItemsTrainDataSet()
# Check the number of training images is as expected
len(traindataset)

In [ ]:
# Create the validation dataset object
valdataset = ItemsValDataSet()
# Check the number of training images is as expected
len(valdataset)

922

In [ ]:
## Created a classifier based on the RESNET50 pretrained model

class ItemClassifier(torch.nn.Module):
    """
    A custom nn.Module class housing the classifier which is
    based on a gpu-derived pretrained resnet50 from NVIDA torchhub
    
    Attributes
    ----------
    None
    
    
    Methods
    -------
    __init__():
        Initialises the classifier and loads the model from the torchhub
        unless it is able to detect a cached instance.
        Replaces the final layer with a 13 class output
    
    forward():
        Initiates the forward pass
        
    """    
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,13)

    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [ ]:
def train(model,traindataloader, valdataloader, epochs):
    """
    Train the modified pretrained resnet50 classifier.
    
    The model weights are written after each epoch to model_evaluation in
    the current working directory
    
    In addition at the end of each epoch:
        Validation Accuracy
        Validation Loss
        Training Accuracy
        Training Loss
    are written to the logs file for the tensorboard to visualise
    performance

    Parameters
    ----------
    model : torch.nn.Module
        The deep learning model to be trained.
    traindataloader : torch.utils.data.DataLoader
        DataLoader for the training dataset.
    valdataloader : torch.utils.data.DataLoader
        DataLoader for the validation dataset.
    epochs : int
        The number of training epochs.

    Returns
    -------
    None
    """    
    optimiser = torch.optim.SGD(model.parameters(), lr=0.01)
    model_path = str(os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S")))
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))

    global_step = 0

    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        model.to(device)
        model.train()
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))
        for inputs, labels in traindataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            loss.backward()
            optimiser.step()
            model_save_dir = str(os.path.join(model_path, 'weights', epoch_combo, 'weights.pt'))
            full_path = str('/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals')
            torch.save({'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimiser.state_dict()},
                  str(os.path.join(full_path, model_save_dir)))

            optimiser.zero_grad()
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]
        training_loss /= len(traindataloader.dataset)
        training_accuracy = tr_num_correct / tr_num_examples
        ## add training performance to tensorboard
        writer.add_scalar('Training Loss', training_loss, global_step)
        writer.add_scalar('Training Accuracy', training_accuracy, global_step)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        for inputs, labels in valdataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            validation_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim =1), dim=1)[1], labels)
            val_num_correct += torch.sum(correct).item()
            val_num_examples += correct.shape[0]
        validation_loss /= len(valdataloader.dataset)
        validation_accuracy = val_num_correct / val_num_examples
        ## add validation performance to tensorboard
        writer.add_scalar('Validation Loss', validation_loss, global_step)
        writer.add_scalar('Validation Accuracy', validation_accuracy, global_step)
        perf_dict = {}
        perf_dict[epoch] = {'training_loss': training_loss,
                            'val_loss': validation_loss,
                            'training_accuracy': tr_num_correct / tr_num_examples,
                            'val_accuracy': val_num_correct / val_num_examples}


        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f},val_accuracy = {:.2f} '.format(epoch, training_loss, validation_loss, tr_num_correct / tr_num_examples,
                                                                                                                             val_num_correct / val_num_examples))
        global_step += 1

In [ ]:
## Create the classifier object from the ItemClassifier class
classifier = ItemClassifier()

Using cache found in /home/jupyter/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [ ]:
## define the layers to unfreeze and then retrain
layers_to_unfreeze = ['layers.2', 'layers.3']

for name, param in classifier.resnet50.named_parameters():
    for layer_name in layers_to_unfreeze:
        if layer_name in name:
            param.requires_grad = True
            break


In [ ]:
## Create the train and validation loaders
## Pass the loaders, classifier and desired number of epochs to the train function define above
train_loader = DataLoader(dataset = traindataset, batch_size=16)
val_loader = DataLoader(dataset = valdataset, batch_size=16)
train(classifier, traindataloader= train_loader, valdataloader= val_loader, epochs=150)


Epoch: 0, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10,val_accuracy = 0.11 
